# Biot equation

In this tutorial we present how to solve the (static) Biot equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$, the flux $q$ and the pressure $p$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, $K$ the permeability of the porous medium and $s_0$ its storativity, and $\alpha$ the Biot-Willis coefficient fot the coupling between the two models.
We want to solve the following problem: find $({u}, {q}, p)$ such that
$$
    \begin{aligned}
         & \nabla \cdot [2 \mu {\epsilon}({u}) + \lambda \nabla \cdot u I
         - \alpha p {I}] = -{b}   \\
         & \mu{q} + {K}\nabla p = {0}                        \\
         & \partial_t (s_0 p + \alpha \nabla \cdot {u}) +
        \nabla \cdot {q} = \psi
    \end{aligned}
    \quad \text{in } \Omega.
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The stress tensor, which can be post-processed from $u$, is given by
$$
    \sigma = 2 \mu \epsilon(u) + \lambda \nabla \cdot u I - \alpha p I
$$

We call the latter $\Delta t$ and write the discrete problem as
$$
    \begin{bmatrix}
        K & 0 & -\alpha D^\top \\
        0 & \Delta t M_q & - \Delta t B^\top \\
        \alpha D & \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
        u^{n+1} \\ q^{n+1} \\ p^{n+1}
    \end{bmatrix}
    =
    \begin{bmatrix}
        b \\ 0 \\ d^n
    \end{bmatrix}
$$
where $d^n = \psi + s_0 M_p p^{n} + \alpha D u^n$ contains the pressure and displacement at the previous time step,
$K$ is the stiffness matrix associated with the elatic problem, $D$ is the coupling between the two physics, $M_q$ is the mass matrix associated to the flux variable, $B$ is the divergence matrix for the flow problem, and $M_p$ is the mass matrix associated with the pressure. All the aforementioned matrices are properly scaled by their physical parameters if not explicitly written. The second row has been multiply by $\Delta t$ to preserve the skew-symmetry of the problem.

## Exercise 3: injection/production problem

An injection problem is when a positive/negative source is impose in a small region of the domain.

For this test case we set $\Omega = [0, 1] \times [0, 1]$, $b = 0$, $\psi = \pm 1$, and the following boundary conditions:
$$ 
\begin{aligned}
    &u = 0 &&\text{ and }&& \nu \cdot q = 0 &&\text{ on } \partial_{bottom} \Omega
\\
&\nu \cdot \sigma = [0, 0]^\top &&\text{ and } && \nu \cdot q = 0 &&\text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega
\\
&\nu \cdot \sigma = [0, 0]^\top &&\text{ and }&& p = 0 &&\text{ on }\partial_{top} \Omega 
\end{aligned}
$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ and Raivar-Thomas for the flux $q$, we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [2]:
mesh_size = 0.05
delta_t = 0.1
num_steps = 10
dim = 2

sd = pg.unit_grid(dim, mesh_size, as_mdg=False)
sd.compute_geometry()

With the following code we set the data and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "biot"

# the physical parameters of the problem, assumed constant
lambda_ = 1
mu = 0.5
alpha = 1
s0 = 1

data = {}
param = {
    "second_order_tensor": pp.SecondOrderTensor(np.ones(sd.num_cells)),
    "lambda": lambda_,
    "mu": mu,
}
pp.initialize_data(sd, data, key, param)

# selection of the boundary conditions
bd_q = sd.tags["domain_boundary_faces"]
bd_q[np.isclose(sd.face_centers[1, :], 1)] = False

bd_u = np.hstack([np.isclose(sd.nodes[1, :], 0)] * dim)

# the scalar source term, +1 to similate injection and -1 for the production
fun = (
    lambda x: -1 * (x[0] > 0.45) * (x[0] < 0.55) * (x[1] > 0.45) * (x[1] < 0.55)
)  # +1 -1

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
# definition of the discretizations
vec_p1 = pg.VecLagrange1(key)
p0 = pg.PwConstants(key)
rt0 = pg.RT0(key)

# dofs for the different variables
dof_u = sd.num_nodes * dim
dof_q = sd.num_faces
dof_p = sd.num_cells
dofs = np.cumsum([dof_u, dof_q, dof_p])

# construction of the block matrices
mass_q = rt0.assemble_mass_matrix(sd)
mass_p = p0.assemble_mass_matrix(sd)
div_q = mass_p @ rt0.assemble_diff_matrix(sd)

sym_sym = vec_p1.assemble_symgrad_symgrad_matrix(sd)
div_div = vec_p1.assemble_div_div_matrix(sd)
div_u = mass_p @ vec_p1.assemble_div_matrix(sd)

stiff_u = 2 * mu * sym_sym + lambda_ * div_div

# fmt: off
# construction of the global problem
spp = sps.block_array([[     stiff_u,              None,   -alpha * div_u.T],
                       [        None,  delta_t * mass_q, -delta_t * div_q.T],
                       [alpha * div_u,  delta_t * div_q,        s0 * mass_p]])


# regroup of the right-hand side
bd = np.hstack((bd_u, bd_q, np.zeros(dof_p, dtype=bool)))
rhs = np.zeros_like(bd, dtype=float)
rhs[-dof_p:] = delta_t * mass_p @ p0.interpolate(sd, fun)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [5]:
# export the solution
save = pp.Exporter(sd, "sol", folder_name="ex3")

# initialization of the solution
u = np.zeros(dof_u)
q = np.zeros(dof_q)
p = np.zeros(dof_p)

# solution of the problem
for n in np.arange(num_steps):
    print(f"Time step {n + 1} of {num_steps}")

    # update of the right-hand side
    rhs_n = rhs.copy()
    rhs_n[-dof_p:] += s0 * mass_p @ p + alpha * div_u @ u

    # solution of the linear system
    ls = pg.LinearSystem(spp, rhs_n)
    ls.flag_ess_bc(bd, np.zeros_like(bd))
    x = ls.solve()

    # split of the solution from the vector x
    u, q, p = np.split(x, dofs[:-1])

    # post-processing for the export
    # compute the cell flow, one vector per cell
    proj_q = rt0.eval_at_cell_centers(sd)
    cell_q = (proj_q @ q).reshape((3, -1))
    cell_p = p0.eval_at_cell_centers(sd) @ p
    node_u = np.hstack((u, np.zeros(sd.num_nodes))).reshape((3, -1))

    # export the solution
    save.write_vtu(
        [
            ("cell_q", cell_q),
            ("cell_p", cell_p),
        ],
        data_pt=[("u", node_u)],
        time_step=n,
    )

Time step 1 of 10
Time step 2 of 10
Time step 3 of 10
Time step 4 of 10
Time step 5 of 10
Time step 6 of 10
Time step 7 of 10
Time step 8 of 10
Time step 9 of 10
Time step 10 of 10


In [8]:
# Consistency check
assert np.isclose(np.linalg.norm(cell_q), 0.24439262818754492)
assert np.isclose(np.linalg.norm(cell_p), 0.11005300364789791)
assert np.isclose(np.linalg.norm(u), 0.014690795255353255)